In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"
from model import SeismicNet
from dense.densenet import FCDenseNet103
from unet.models import NestedUNet

In [2]:
!nvidia-smi

Tue Jun  2 14:05:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:61:00.0 Off |                  Off |
| N/A   39C    P0    36W / 250W |   2031MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:DB:00.0 Off |                  Off |
| N/A   48C    P0    44W / 250W |   4381MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
#net = SeismicNet(1, 1)
#net = FCDenseNet103(1)
net = NestedUNet(1, 1)
net = net.float()

In [4]:
def dice_loss(pred,target):
    numerator = 2 * torch.sum(pred * target)
    denominator = torch.sum(pred + target)
    return 1 - (numerator + 1) / (denominator + 1)


In [5]:
def dice_coef(pred,target):
    numerator = 2 * torch.sum(pred * target)
    denominator = torch.sum(pred + target)
    return (numerator + 1) / (denominator + 1)


In [6]:
import numpy as np

In [7]:
import torch

img = np.zeros((1, 1, 256, 256))
img = torch.from_numpy(img)
out = net(img.float())



In [7]:
def tversky_loss(true, logits, alpha, beta, eps=1e-7):
    """Computes the Tversky loss [1].
    Args:
        true: a tensor of shape [B, H, W] or [B, 1, H, W].
        logits: a tensor of shape [B, C, H, W]. Corresponds to
            the raw output or logits of the model.
        alpha: controls the penalty for false positives.
        beta: controls the penalty for false negatives.
        eps: added to the denominator for numerical stability.
    Returns:
        tversky_loss: the Tversky loss.
    Notes:
        alpha = beta = 0.5 => dice coeff
        alpha = beta = 1 => tanimoto coeff
        alpha + beta = 1 => F beta coeff
    References:
        [1]: https://arxiv.org/abs/1706.05721
    """
    num_classes = logits.shape[1]
    if num_classes == 1:
        true_1_hot = torch.eye(num_classes + 1)[true.squeeze(1)]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        true_1_hot_f = true_1_hot[:, 0:1, :, :]
        true_1_hot_s = true_1_hot[:, 1:2, :, :]
        true_1_hot = torch.cat([true_1_hot_s, true_1_hot_f], dim=1)
        pos_prob = torch.sigmoid(logits)
        neg_prob = 1 - pos_prob
        probas = torch.cat([pos_prob, neg_prob], dim=1)
    else:
        true_1_hot = torch.eye(num_classes)[true.squeeze(1)]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        probas = F.softmax(logits, dim=1)
    true_1_hot = true_1_hot.type(logits.type())
    dims = (0,) + tuple(range(2, true.ndimension()))
    intersection = torch.sum(probas * true_1_hot, dims)
    fps = torch.sum(probas * (1 - true_1_hot), dims)
    fns = torch.sum((1 - probas) * true_1_hot, dims)
    num = intersection
    denom = intersection + (alpha * fps) + (beta * fns)
    tversky_loss = (num / (denom + eps)).mean()
    return (1 - tversky_loss)


In [8]:
import albumentations as A



In [9]:
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from torch.nn import DataParallel

A.ShiftScaleRotate(rotate_limit=20, )
size = (256, 256)
light = A.Compose([
     #A.RandomSizedCrop((450, 450), 512, 512),     
     A.ShiftScaleRotate(rotate_limit=20),
     A.Blur(),
     A.GaussNoise(),
     A.Resize(size[0], size[1])
],p=1)

In [10]:
import cv2
import glob, os
import torch
import pandas as pd

class CovidDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=light):
            self.data = pd.read_csv(csv_file)
            self.root_dir = root_dir
            self.transform = transform
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 1])
        msk_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 2])        
        try:
            image = cv2.imread(img_name, 0)
            mask = cv2.imread(msk_name, 0) // 255
        except:
            print(img_name, msk_name)
            pass
        
        if self.transform is not None:
            aug = self.transform(image=image, mask=mask)
        else:
            aug = dict()
            aug['image'] = image
            aug['mask'] = mask
        
        
        return torch.from_numpy(aug['image'][np.newaxis]), torch.from_numpy(aug['mask'][np.newaxis])

In [11]:
batch_size = 15
cov_dataset_train_bad = CovidDataset(csv_file='./data_train/train_bad.csv',
                                     root_dir='./data_train/bad/')
cov_dataset_val_bad = CovidDataset(csv_file='./data_train/val_bad.csv',
                                     root_dir='./data_train/bad/', transform=None)
cov_dataset_train_good = CovidDataset(csv_file='./data_train/train_good.csv',
                                     root_dir='./data_train/good/')
cov_dataset_val_good = CovidDataset(csv_file='./data_train/val_good.csv',
                                     root_dir='./data_train/good/', transform=None)

In [12]:
dataloader_train_bad = DataLoader(cov_dataset_train_bad, batch_size=batch_size,
                        shuffle=True, num_workers=4)
dataloader_val_bad = DataLoader(cov_dataset_val_bad, batch_size=batch_size,
                        shuffle=True, num_workers=4)
dataloader_train_good = DataLoader(cov_dataset_train_good, batch_size=batch_size,
                        shuffle=True, num_workers=4)
dataloader_val_good = DataLoader(cov_dataset_val_good, batch_size=batch_size,
                        shuffle=True, num_workers=4)

In [13]:
!rm -rf /cv/volume/experiments/covid_nested_v5_new_data

In [14]:
from collections import OrderedDict
new_state_dict = OrderedDict()

In [18]:
import torch
state_dict = torch.load('save/nested_v3/40.pth.tar')

In [16]:
for k, v in state_dict.items():
    new_state_dict[k[7:]] = v

In [19]:
net.load_state_dict(state_dict)

<All keys matched successfully>

In [14]:
net = net.cuda()

In [15]:
import torch.optim as optim
import torch.nn as nn
from tensorboardX import SummaryWriter
#optimizer = optim.RMSprop(net.parameters(), lr=0.01, weight_decay=1e-8, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.01)
name = 'covid_nested_v5_new_data'
exp_path = '/cv/volume/experiments/{}'.format(name)
writer = SummaryWriter(exp_path)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)
#criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(7))
net = DataParallel(net, device_ids=[0, 1]).train()
net = net.cuda()



In [17]:
name = 'covid_nested_v5_new_data'
exp_path = '/cv/volume/experiments/{}'.format(name)

In [24]:
net.load_state_dict(torch.load('save/nested_v3/40.pth.tar'))

<All keys matched successfully>

In [16]:
for i, data in enumerate(dataloader_val_bad):
    break
    continue

In [18]:
data[1].max()

tensor(1, dtype=torch.uint8)

In [13]:
import matplotlib.pyplot as plt
n = 3
plt.imshow(data[0].squeeze().cpu().detach().numpy()[n])

NameError: name 'data' is not defined

In [ ]:
plt.imshow(data[1].squeeze().cpu().detach().numpy()[n])

In [20]:
data[1].squeeze().cpu().detach().numpy().max()

1

In [20]:
len(dataloader_val_bad)

38

In [24]:
import torch.nn as nn
global_step = 0
s = nn.Sigmoid()

In [ ]:
import torchvision.utils as vutils
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

global_step = 0
s = nn.Sigmoid()
best_auc = 0
from tqdm import tqdm_notebook as tqdm 
for epoch in tqdm(range(100)):
    running_loss = 0.0
    value = 0.0
    
    net.train()
    i = 0
    bad_iter = iter(dataloader_train_bad)
    good_iter = iter(dataloader_train_good)
    
    
    for step in range(len(dataloader_train_bad)):
        try:
            img_bad, mask_bad = next(bad_iter)
            img_good, mask_good = next(good_iter)
        except:
            dataloader_train_good = DataLoader(cov_dataset_train_good, batch_size=batch_size,
                        shuffle=True, num_workers=4)
            dataloader_train_bad = DataLoader(cov_dataset_train_bad, batch_size=batch_size,
                        shuffle=True, num_workers=4)
            break
        i += 1
        
        inputs = torch.cat([img_bad, img_good]).cuda().float()
        masks = torch.cat([mask_bad, mask_good]).cuda().float()
            
        optimizer.zero_grad()

        outputs = net(inputs)
        
        loss = criterion(outputs, masks)
        #loss = tversky_loss(masks.long(), outputs, 0.7, 0.3)
        outputs = s(outputs)        
        outputs[outputs > 0.6] = 1
        outputs[outputs <= 0.6] = 0
        
        if global_step % 50 == 0:
            x = torch.cat([img_bad[:2].detach().cpu().float(), mask_bad[:2].detach().cpu().float(), outputs[:2].detach().cpu().float()])
            x = vutils.make_grid(x, normalize=True, scale_each=True)
            writer.add_image('train/Image', x, global_step)  # Tensor
            
        dice = dice_loss(outputs, masks)
        
        loss.backward()
        optimizer.step()

        writer.add_scalar('train/loss_CE', loss.item(), global_step)
        writer.add_scalar('train/DICE', 1 - dice.item(), global_step)
        global_step += 1
        print(f"[TRAIN][BATCH {i}/{len(dataloader_train_bad)}] {loss.item()} | {dice.item()}\r", end="")   
    
    net.eval()
    dice = 0
    running_loss = 0
    y_pred = []
    y_true = []
    with torch.no_grad():
        for i, data in enumerate(dataloader_val_bad):
            inputs = data[0].cuda().float()
            masks = data[1].cuda().float()

            outputs = net(inputs)
            loss = criterion(outputs, masks).detach()
            #loss = tversky_loss(masks.long(), outputs, 0.7, 0.3).detach()
            running_loss += loss.item()
            outputs = s(outputs)
            
            
            outputs[outputs > 0.75] = 1
            outputs[outputs <= 0.75] = 0
            
            if len(outputs[outputs != 0]) > 0:
                y_pred.append(1)
            else:
                y_pred.append(0)
            y_true.append(1)
            
            if i == 0:
                x = torch.cat([inputs[:2].detach().cpu().float(), masks[:2].detach().cpu().float(), outputs[:2].detach().cpu().float()])
                x = vutils.make_grid(x, normalize=True, scale_each=True)
                writer.add_image('val/image_bad', x, epoch)  # Tensor
                
            dice += dice_coef(outputs, masks)

            print(f"[VALID_BAD][BATCH {i}/{len(dataloader_val_bad)}] {running_loss / (i + 1)} | {dice / (i + 1)}\r", end="")

        for i, data in enumerate(dataloader_val_good):
            inputs = data[0].cuda().float()
            masks = data[1].cuda().float()

            outputs = net(inputs)
            loss = criterion(outputs, masks).detach()
            #loss = tversky_loss(masks.long(), outputs, 0.7, 0.3).detach()
            running_loss += loss.item()
            outputs = s(outputs)
            
            outputs[outputs > 0.75] = 1
            outputs[outputs <= 0.75] = 0
            
            if len(outputs[outputs != 0]) > 0:
                y_pred.append(1)
            else:
                y_pred.append(0)
            y_true.append(0)
            dice += dice_coef(outputs, masks)
            
            if i == 0:
                x = torch.cat([inputs[:2].detach().cpu().float(), masks[:2].detach().cpu().float(), outputs[:2].detach().cpu().float()])
                x = vutils.make_grid(x, normalize=True, scale_each=True)
                writer.add_image('val/image_good', x, epoch)  # Tensor

            print(f"[VALID_GOOD][BATCH {i}/{len(dataloader_val_bad)}] {running_loss / (i + 1)} | {dice / (i + 1)}\r", end="")
        
        accuracy = accuracy_score(y_true, y_pred)
        
        writer.add_scalar('val/loss_CE', running_loss / (len(dataloader_val_bad) + len(dataloader_val_good)), epoch)
        writer.add_scalar('val/dice_loss', dice / (len(dataloader_val_bad) + len(dataloader_val_good)), epoch)
        writer.add_scalar('val/accuracy', accuracy, epoch)
        #scheduler.step(running_loss / (len(dataloader_val_bad) + len(dataloader_val_good)))
        writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], epoch)
        torch.save(net.module.state_dict(), 'save/{}/{}.pth.tar'.format(name, epoch))
        if accuracy > best_auc:
            torch.save(net.module.state_dict(), 'save/{}/best.pth.tar'.format(name))
            best_auc = accuracy

    
    
        
        
        
        
    
    
    

/miniconda/envs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [1]:
from sklearn.metrics import roc_auc_score

roc_auc_score([0, 0, 1], [1, 0, 0])

0.25

In [17]:
mask_bad.dtype

torch.uint8

In [18]:
img_bad.dtype

torch.uint8

In [19]:
outputs.dtype

torch.float32

In [ ]:
out_

In [19]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [20]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [21]:
writer.close()

In [22]:
torch.save(net.state_dict(), './save.pth.tar')

In [12]:
#net.cuda()
#print('cuda')
import torch

a = [0, 0, 0]
b = [1, 0, 0]

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(a, b)

/miniconda/envs/cv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/miniconda/envs/cv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/miniconda/envs/cv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/miniconda/envs/cv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/miniconda/envs/cv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompat

0.6666666666666666

In [36]:
net.eval()
print('eval')

eval


In [37]:
import numpy as np
for i, data in enumerate(dataloader_val_bad):
    inputs = data[0].cuda().float()
    masks = data[1].cuda().float()

    outputs = net(inputs).detach()
    break

In [38]:
inputs.shape

torch.Size([10, 1, 512, 512])

In [39]:
out = s(outputs)

In [40]:
i = 3
img = out[i].squeeze().detach().cpu().numpy()
mask = masks[i].squeeze().detach().cpu().numpy()
mask.shape

(512, 512)

In [41]:
np.where(img.nonzero())

(array([0, 0, 0, ..., 1, 1, 1]),
 array([   512,    513,    514, ..., 262141, 262142, 262143]))

In [42]:
np.where(mask.nonzero())

(array([0, 0, 0, ..., 1, 1, 1]),
 array([   0,    1,    2, ..., 4531, 4532, 4533]))

In [43]:
img

array([[3.0556468e-05, 6.5137465e-06, 5.1085622e-06, ..., 4.4201506e-06,
        5.1814486e-06, 2.6235362e-05],
       [6.2028371e-06, 4.7883032e-07, 2.9595981e-07, ..., 2.5399638e-07,
        4.1503981e-07, 5.5252667e-06],
       [4.7944768e-06, 2.7802477e-07, 1.6657127e-07, ..., 1.2052726e-07,
        2.3460022e-07, 4.3861000e-06],
       ...,
       [2.8513573e-06, 1.3606058e-07, 7.5703419e-08, ..., 6.3742348e-08,
        1.4557675e-07, 3.2444168e-06],
       [2.9685214e-06, 1.6397068e-07, 9.0943622e-08, ..., 9.7633034e-08,
        2.0748362e-07, 3.7996001e-06],
       [1.5303340e-05, 2.3283435e-06, 1.6350239e-06, ..., 1.7547600e-06,
        2.7224407e-06, 1.9645160e-05]], dtype=float32)